# 02. 임베딩(Embedding)과 RAG

이 노트북에서 다루는 내용:
- 텍스트를 위한 임베딩 생성
- pgvector를 사용하여 PostgreSQL에 임베딩 저장
- 시맨틱 검색(Semantic Search) 수행
- 간단한 RAG(검색 증강 생성, Retrieval-Augmented Generation) 시스템 구축

## 1. 라이브러리 임포트

In [ ]:
import sys
sys.path.append('/workspace')

from src.utils.db_utils import DatabaseConnection
from src.utils.embedding_utils import (
    EmbeddingGenerator,
    chunk_text,
    store_document_with_embedding,
    search_similar_documents
)
import pandas as pd

print("✓ Libraries imported successfully")

## 2. 컴포넌트 초기화

In [ ]:
# 데이터베이스 초기화 connection
db = DatabaseConnection()

# 임베딩 생성기 초기화
embedder = EmbeddingGenerator()
print(f"✓ Using embedding model: {embedder.model_name}")
print(f"✓ Embedding dimension: {embedder.embedding_dim}")

## 3. 임베딩 생성

In [ ]:
# 임베딩 생성 테스트
test_texts = [
    "What is the total revenue from sales?",
    "Show me all employees in the engineering department",
    "List all active projects"
]

embeddings = embedder.generate_embeddings(test_texts)
print(f"Generated {len(embeddings)} embeddings")
print(f"Each embedding has {len(embeddings[0])} dimensions")
print(f"\nFirst embedding (first 10 values): {embeddings[0][:10]}")

## 4. 시맨틱 유사도 테스트

In [ ]:
# 쿼리 간 유사도 비교
query1 = "Show me the revenue data"
query2 = "What is the total sales amount?"
query3 = "List all employees"

emb1 = embedder.generate_embedding(query1)
emb2 = embedder.generate_embedding(query2)
emb3 = embedder.generate_embedding(query3)

sim_1_2 = embedder.cosine_similarity(emb1, emb2)
sim_1_3 = embedder.cosine_similarity(emb1, emb3)

print(f"Similarity between '{query1}' and '{query2}': {sim_1_2:.4f}")
print(f"Similarity between '{query1}' and '{query3}': {sim_1_3:.4f}")
print("\n💡 Higher scores indicate more semantic similarity")

## 5. 임베딩으로 용어집 업데이트

In [ ]:
# 임베딩이 없는 기존 용어집 항목 가져오기
query = "SELECT * FROM lexicon WHERE embedding IS NULL"
lexicon_entries = db.execute_query(query)

print(f"Found {len(lexicon_entries)} lexicon entries without embeddings")

# 임베딩 생성 및 업데이트
for entry in lexicon_entries:
    # 더 나은 시맨틱 표현을 위해 용어와 정의 결합
    text = f"{entry['term']}: {entry['definition']}"
    embedding = embedder.generate_embedding(text)
    
    # 데이터베이스 업데이트
    update_query = "UPDATE lexicon SET embedding = %s WHERE lexicon_id = %s"
    conn = db.get_connection()
    try:
        with conn.cursor() as cursor:
            cursor.execute(update_query, (embedding, entry['lexicon_id']))
        conn.commit()
    finally:
        conn.close()
    
    print(f"✓ Updated embedding for: {entry['term']}")

print("\n✓ All lexicon entries updated with embeddings")

## 6. 임베딩으로 문서 업데이트

In [ ]:
# 임베딩이 없는 기존 문서 가져오기
query = "SELECT * FROM documents WHERE embedding IS NULL"
documents = db.execute_query(query)

print(f"Found {len(documents)} documents without embeddings")

# 임베딩 생성 및 업데이트
for doc in documents:
    # 임베딩을 위해 제목과 내용 사용
    text = f"{doc['title']}: {doc['content']}"
    embedding = embedder.generate_embedding(text)
    
    # 데이터베이스 업데이트
    update_query = "UPDATE documents SET embedding = %s WHERE document_id = %s"
    conn = db.get_connection()
    try:
        with conn.cursor() as cursor:
            cursor.execute(update_query, (embedding, doc['document_id']))
        conn.commit()
    finally:
        conn.close()
    
    print(f"✓ Updated embedding for: {doc['title']}")

print("\n✓ All documents updated with embeddings")

## 7. 시맨틱 검색 - 용어집

In [ ]:
# 관련 용어집 항목 검색
search_query = "How do I convert natural language to SQL?"
query_embedding = embedder.generate_embedding(search_query)

sql = """
SELECT 
    term,
    definition,
    category,
    1 - (embedding <=> %s::vector) as similarity
FROM lexicon
WHERE embedding IS NOT NULL
ORDER BY embedding <=> %s::vector
LIMIT 3
"""

results = db.execute_query(sql, (query_embedding, query_embedding))

print(f"Search query: '{search_query}'")
print("\nTop 3 relevant lexicon entries:\n")
for i, result in enumerate(results, 1):
    print(f"{i}. {result['term']} (similarity: {result['similarity']:.4f})")
    print(f"   Category: {result['category']}")
    print(f"   Definition: {result['definition'][:100]}...")
    print()

## 8. 시맨틱 검색 - 문서

In [ ]:
# 유틸리티 함수를 사용하여 관련 문서 검색
search_query = "How to implement RAG system?"
results = search_similar_documents(db, search_query, limit=3)

print(f"Search query: '{search_query}'")
print("\nTop 3 relevant documents:\n")
for i, (doc_id, title, content, similarity) in enumerate(results, 1):
    print(f"{i}. {title} (similarity: {similarity:.4f})")
    print(f"   Content: {content[:150]}...")
    print()

## 9. 임베딩과 함께 새 문서 추가

In [ ]:
# 자동 임베딩과 함께 새 문서 추가
new_doc = {
    'title': 'Text2SQL Implementation Tips',
    'content': '''Key tips for implementing text2sql:
    1. Always include database schema in your prompt
    2. Use few-shot examples to improve accuracy
    3. Validate generated SQL before execution
    4. Handle errors gracefully
    5. Log all queries for monitoring and improvement
    6. Consider using semantic similarity for schema retrieval
    7. Test with various natural language phrasings
    ''',
    'doc_type': 'Guide',
    'metadata': {'category': 'text2sql', 'author': 'system'}
}

doc_id = store_document_with_embedding(
    db,
    title=new_doc['title'],
    content=new_doc['content'],
    doc_type=new_doc['doc_type'],
    metadata=new_doc['metadata']
)

print(f"✓ Added new document with ID: {doc_id}")
print(f"  Title: {new_doc['title']}")

## 10. 간단한 RAG 컨텍스트 구축

In [ ]:
def build_rag_context(query: str, max_docs: int = 3) -> str:
    """
    Build RAG context by retrieving relevant documents
    """
    # Search for relevant documents
    results = search_similar_documents(db, query, limit=max_docs)
    
    # Build context string
    context = "Relevant Information:\n\n"
    for i, (doc_id, title, content, similarity) in enumerate(results, 1):
        context += f"{i}. {title}\n{content}\n\n"
    
    return context

# Test RAG context building
user_query = "What are best practices for text2sql?"
rag_context = build_rag_context(user_query)

print(f"User Query: {user_query}")
print("\n" + "="*60)
print("RAG Context (to be used in LLM prompt):")
print("="*60)
print(rag_context)

## 11. 텍스트 청킹(Chunking) 예제

In [ ]:
# 긴 텍스트 청킹 예제
long_text = """
Text2SQL is a technology that converts natural language queries into SQL statements.
It uses large language models to understand the user's intent and generate appropriate
database queries. The process typically involves: 1) Understanding the database schema,
2) Parsing the natural language query, 3) Mapping entities to database tables and columns,
4) Generating the SQL query, and 5) Validating and executing the query.

Best practices include providing clear schema information, using few-shot examples,
implementing proper error handling, and logging all queries for monitoring. It's also
important to validate generated SQL to prevent injection attacks and ensure query safety.

RAG can enhance text2sql by retrieving relevant schema information and examples.
This helps the model generate more accurate queries by providing additional context.
""" * 3  # Repeat to make it longer

chunks = chunk_text(long_text, chunk_size=200, overlap=50)

print(f"Original text length: {len(long_text)} characters")
print(f"Number of chunks: {len(chunks)}")
print("\nChunks:")
for i, chunk in enumerate(chunks, 1):
    print(f"\nChunk {i} ({len(chunk)} chars):")
    print(chunk[:100] + "...")

## 요약

이 노트북에서 배운 내용:
- ✓ Sentence Transformers를 사용한 임베딩 생성 방법
- ✓ pgvector를 사용하여 PostgreSQL에 임베딩을 저장하는 방법
- ✓ 시맨틱 유사도 검색 수행 방법
- ✓ 간단한 RAG 시스템 구축 방법
- ✓ 처리를 위한 텍스트 청킹 방법

다음 단계: `03_text2sql_basic.ipynb`로 이동하여 Text2SQL 실습을 시작합니다.